In [88]:
# Python import 
import glob
import os
import numpy as np
import math 
import pandas as pd 
import time
import shutil
# Skimage, Image processing
import skimage
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from skimage import color, io
# Pillow
from PIL import Image, ImageDraw

import collections
import SimpleITK as sitk

In [89]:
series_uid = "1.3.6.1.4.1.14519.5.2.1.6279.6001.121824995088859376862458155637"
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])
def irc2xyz(coord_irc, origin_xyz, vxSize_xyz, direction_a):
    cri_a = np.array(coord_irc)[::-1]
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coords_xyz = (direction_a @ (cri_a * vxSize_a)) + origin_a
    # coords_xyz = (direction_a @ (idx * vxSize_a)) + origin_a
    return XyzTuple(*coords_xyz)

Get gT

In [90]:
df_node = pd.read_csv("data/annotations.csv")
def circle_gt(series_uid, origin, spacing):
    # for fcount, img_file in enumerate(tqdm(file_list)):
    mini_df = df_node[df_node["seriesuid"]==series_uid]  #get all nodules associate with file
    ans = []
    if mini_df.shape[0] > 0:  # some files may not have a nodule--skipping those
        for node_idx, cur_row in mini_df.iterrows():
            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            diam = cur_row["diameter_mm"]

            # Get v_center
            center = np.array([node_x, node_y, node_z])   # nodule center
            v_center = np.rint(abs(center-origin)/spacing)  # nodule center in voxel space (still x,y,z ordering)
            vx, vy, vz = v_center
            x1 = int(vx-diam)
            y1 = int(vy-diam)
            x2 = int(vx+diam)
            y2 = int(vy+diam)
            ans.append( (vz, x1, y1, x2, y2) )
            # ans.append( v_center ) # [(vx, vy, vz)]
    return ans


In [131]:
o_seriesuid = []
o_coordX = []
o_coordY = []
o_coordZ = []
o_class = []

mhd_list = glob.glob("data/train/*.mhd")
shutil.rmtree("./seg_result/", ignore_errors=True)
os.mkdir("./seg_result/")

# Very big tumor
# 
# This guy have a very weird lung
# 1.3.6.1.4.1.14519.5.2.1.6279.6001.614147706162329660656328811671
# My favoriate
# 1.3.6.1.4.1.14519.5.2.1.6279.6001.121824995088859376862458155637

for mhd_idx, mhd_fn in enumerate(mhd_list):

    series_uid = os.path.split(mhd_fn)[1][:-4]
    # if series_uid != "1.3.6.1.4.1.14519.5.2.1.6279.6001.280072876841890439628529365478":
    #     continue
    
    os.mkdir(f"./seg_result/{series_uid}")

    ct_mhd = sitk.ReadImage(f"data/train/{series_uid}.mhd")
    origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
    vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
    direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    nodule_a = np.load(f"npy/{series_uid}_nodule_a.npy")
    origin, spacing = np.load(f"npy/{series_uid}_ori_space_a.npy")

    gt_list = circle_gt(series_uid,origin,spacing)

    a = nodule_a
    a += 1000
    a *= (255.0/1000)
    a = a.astype('uint8')
    # Binarize
    a[a > 0] = 255

    r_candidate_list = []
    seg_a_viz = a.copy()
    for seg_idx in range(a.shape[0]):
        try:
            seg_a = a[seg_idx] + a[seg_idx-1] + a[seg_idx+1]
        except IndexError as e :
            seg_a = a[seg_idx]
        
        seg_a[seg_a > 0] = 255

        label_scan, num  = label(seg_a, background=0, connectivity=1, return_num = True)
        # print(f"Number of connected region: {num}")

        props = regionprops(label_scan)

        # For viz 
        seg_a_viz[seg_idx] = seg_a

        r_candidate = []
        for r in props:
            # Elimiate too small area  and too large area
            if r.area >= 10 and r.area <= 800 and r.eccentricity < 0.9 and r.extent > 0.6:
                r_candidate.append(r)

        r_candidate_list.append(r_candidate)

    # Refine RP
    r_candidate_refined = []
    for seg_idx in range(seg_a_viz.shape[0]):
        r_list = []
        for r in r_candidate_list[seg_idx]:
            x_l, y_l = r.centroid
            a_l = r.area

            # Find the largest area in afterwards images
            done = False
            for seg_idx_tmp in range(seg_idx+1, seg_a_viz.shape[0], 1):
                is_consecutive = False
                if seg_idx_tmp >= seg_a_viz.shape[0]:
                    break
                
                for r_tmp in r_candidate_list[seg_idx_tmp]: # Search all candidate 
                    x_tmp, y_tmp = r_tmp.centroid
                    if abs(x_tmp - x_l) + abs(y_tmp - y_l) < 6: # Consider as the same object
                        is_consecutive = True
                        # Found the object in afterward image
                        if r_tmp.area >= a_l:
                            # The search is done and we don't need to keep it
                            done = True
                            break
                        else:
                            # Continue to search
                            # Replace x_c, y_c and keep searching next frame
                            x_l = x_tmp
                            y_l = y_tmp
                            a_l = r_tmp.area
                            break
                if done:
                    break
                if not is_consecutive:
                    # Look back one frame to check there's no object before
                    
                    is_replicate = False
                    if seg_idx != 0:
                        x_l, y_l = r.centroid
                        for r_tmp in r_candidate_refined[seg_idx-1]:
                            x_tmp, y_tmp = r_tmp.centroid
                            if abs(x_tmp - x_l) + abs(y_tmp - y_l) < 6: # Consider as the same object
                                is_replicate = True
                                break
                    if not is_replicate:
                        r_list.append(r) # Done search, 
                    
                    # r_list.append(r) # Done search, 
                    break
        r_candidate_refined.append(r_list)

    n_region = 0
    for x in r_candidate_refined:
        n_region += len(x)
    print(f"r_candidate_refined number of region : {n_region}")

    n_region = 0
    for x in r_candidate_list:
        n_region += len(x)
    print(f"r_candidate_list number of region : {n_region}")
    r_candidate_list = r_candidate_refined

    # Delete region proposal if there's a GT
    # TODO This will only be used, when outputing candidate.csv
    for vz, x1, y1, x2, y2 in gt_list:
        x_gt = (x1+x2)/2
        y_gt = (y1+y2)/2
        for clean_idx in range(int(vz)-3, int(vz)+3, 1):
            for i, r in enumerate(r_candidate_list[clean_idx]):
                x_c, y_c = r.centroid
                if abs(x_gt - x_c) + abs(y_gt - y_c) < 6: # Consider as the same object
                    del r_candidate_list[clean_idx][i]
                    break

    for seg_idx in range(seg_a_viz.shape[0]):
        # Visualization 
        img = Image.fromarray(seg_a_viz[seg_idx])
        img = img.convert("RGB")
        
        # Draw Ground true 
        for vz, x1, y1, x2, y2 in gt_list:
            if int(vz) == seg_idx:
                draw = ImageDraw.Draw(img)
                draw.ellipse((x1, y1, x2, y2), outline=(0, 255, 0))
        
        # Draw Region Proposal
        for r in r_candidate_list[seg_idx]:
            x_c, y_c = r.centroid
            y1, x1, y2, x2 = r.bbox
            draw = ImageDraw.Draw(img)
            # draw.ellipse((x1, y1, x2, y2), outline=(255, 0, 0))
            draw.rectangle((x1, y1, x2, y2), outline='red', width =1 )

        # Output candidate, class = 0
        for r in r_candidate_list[seg_idx]:
            x_c, y_c = r.centroid
            # Output candidate
            xyz = irc2xyz( IrcTuple(seg_idx, x_c, y_c), origin_xyz, vxSize_xyz, direction_a)
            # f"{round(xyz.x, 2)},{round(xyz.x, 2)},{round(xyz.x, 2)} "

            o_seriesuid.append(series_uid)
            o_coordX.append( round(xyz.x, 9) )
            o_coordY.append( round(xyz.y, 9) )
            o_coordZ.append( round(xyz.z, 9) )
            o_class.append(0)
        
        # Viz
        # for vz, x1, y1, x2, y2 in gt_list:
        #     if abs(seg_idx - vz) < 5:
        #         display(img)
        #         for r in r_candidate_list[seg_idx]:
        #             print(f"(x, y, a) = {r.centroid[0], r.centroid[1], r.area}")
        #         # img.save(f'seg_result/{series_uid}/{seg_idx}.png')

    # Output gt to candidate, class = 1
    mini_df = df_node[df_node["seriesuid"]==series_uid]
    for node_idx, cur_row in mini_df.iterrows():
        o_seriesuid.append(series_uid)
        o_coordX.append( cur_row["coordX"] )
        o_coordY.append( cur_row["coordY"] )
        o_coordZ.append( cur_row["coordZ"] )
        o_class.append(1)

    print(f"Processed {mhd_idx+1}/{len(mhd_list)}")

df_output = pd.DataFrame()
df_output['series_uid'] = o_seriesuid
df_output['coordX'] = o_coordX
df_output['coordY'] = o_coordY
df_output['coordZ'] = o_coordZ
df_output['class'] = o_class
df_output.to_csv("seg.csv", index = False)

r_candidate_refined number of region : 194
r_candidate_list number of region : 266
Processed 1/267


KeyboardInterrupt: 